# Linear Regression Model Training Service

Beskriv formål her

Tasks:
1. Retrieving test data from REST API (update URL needed)
2. Preprocessing data
3. Training LinearRegression with scikit-learn
4. Evaluating model
5. Hyperparameter tuning for model
6. Evaluating model on unseen test data
7. Exporting model as .onnx (to do)

### 1. Retrieving data from REST API

In [ ]:
import requests
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

# === 1. Fetch training data ===
# TODO: Replace with actual API endpoint
response = requests.get('http://api-url/training-data')
data_json = response.json()
data = pd.DataFrame(data_json)


### 2. Preprocessing data

In [ ]:
# === 2. Split into training, validation, and test sets ===
X = data.drop('y', axis=1)
y = data['y']

# First, split 80% temporary set and 20% test set
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Then split temporary set into 60% training and 20% validation
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)

print(f'Training set size: {X_train.shape[0]} samples')
print(f'Validation set size: {X_val.shape[0]} samples')
print(f'Test set size: {X_test.shape[0]} samples')

# Scale features using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Hyperparameter tuning with GridSearchCV using Ridge Regression
param_grid = {
    'alpha': [0.01, 0.1, 1, 10, 100],
}

grid_search = GridSearchCV(
    Ridge(),
    param_grid,
    scoring='neg_mean_squared_error',
    cv=5
)

grid_search.fit(X_train_scaled, y_train)
print("Best Ridge parameters:", grid_search.best_params_)

### 4. Evaluate model on validation set

In [ ]:
# Evaluating model on validation set
best_model = grid_search.best_estimator_

y_val_pred = best_model.predict(X_val_scaled)
val_rmse = mean_squared_error(y_val, y_val_pred, squared=False)
val_r2 = r2_score(y_val, y_val_pred)

print("Validation performance:")
print(f"RMSE: {val_rmse:.2f}")
print(f"R²: {val_r2:.2f}")

### 6. Evaluate model on test set

In [ ]:
# Evaluating model based on unseen test data

y_test_pred = best_model.predict(X_test_scaled)
test_rmse = mean_squared_error(y_test, y_test_pred, squared=False)
test_r2 = r2_score(y_test, y_test_pred)

print("Test performance:")
print(f"RMSE: {test_rmse:.2f}")
print(f"R²: {test_r2:.2f}")

### 7. Export model as ONNX

In [ ]:
initial_type = [('input', FloatTensorType([None, X_train_scaled.shape[1]]))]
onnx_model = convert_sklearn(best_model, initial_types=initial_type)

with open("../app/model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

print("Model exported as ONNX to ../app/model.onnx")